# mapboxgl-jupyter

Testing [mapboxgl-jupyter](https://github.com/mapbox/mapboxgl-jupyter/) for creating maps in Jupyter notebooks. The repo itself provides some nice [examples](https://github.com/mapbox/mapboxgl-jupyter/tree/master/examples/notebooks).

In [1]:
import geojson
import geopandas as gpd
import os
import requests
import zipfile

from mapboxgl.viz import *
from mapboxgl.utils import *

In [2]:
token = os.getenv('MAPBOX_ACCESS_TOKEN')

## Download and cache data

Get Shapefile of health sites in Colombia and prepare a GeoJSON for mapping.

In [3]:
data_url = 'https://healthsites.io/media/shapefiles/Colombia_shapefile.zip'
data_path = 'data/colombia.zip'

if not os.path.exists(data_path):
    r = requests.get(data_url)
    with open(data_path, 'wb') as f:
        f.write(r.content)        

    with zipfile.ZipFile(data_path, 'r') as f:
        f.extractall(os.path.splitext(data_path)[0])    

data_gdf_all_columns = gpd.read_file('data/colombia/colombia.shp')
data_gdf = data_gdf_all_columns[['name', 'type', 'geometry']]
data = geojson.loads(data_gdf.to_json())

## Color health sites by type

In [4]:
category_color_stops = [
    ['hospital', 'rgb(211,47,47)'],  
    ['clinic', 'rgb(2,136,209)'], 
]

# Initialize CircleViz with categorical measure data
viz = CircleViz(data, access_token=token, height='500px')

# Marker color-related attributes
viz.color_property = 'type'
viz.default_color = 'grey'
viz.color_function_type = 'match'
viz.color_stops = category_color_stops

# Set center, zoom and render map
viz.center = (-74.2973, 4.5709)
viz.zoom = 4
viz.show() 

## Heatmap

In [5]:
heatmap_color_stops = create_color_stops([0.01, 0.25, 0.5, 0.75, 1], colors='OrRd')
heatmap_radius_stops = [[0, 1], [15, 20]]  # increase radius with zoom

viz = HeatmapViz(data, 
                 access_token=token,
                 weight_stops=[[1, 1]],
                 color_stops=heatmap_color_stops,
                 radius_stops=heatmap_radius_stops,
                 opacity=0.8,
                 center=(-74.2973, 4.5709),
                 zoom = 4
                )
viz.show()